# Construindo uma enciclopedia

Este projeto consiste na criação de uma enciclopédia utilizando conceitos de processamento de linguagem natural, neste são implementados etapas como extração de palavras de documentos do corpus reuters, extração de conteúdos da api do wikipedia referente a cada palavra, processamento desses conteúdos retirando caracteres e textos desnecessários, e treinamento de um modelo para selecionar as melhores descrições das palavras com análises de perplexidade.

## Importa dependências

In [19]:
# importe de bibliotecas
import re
import nltk
from nltk.corpus import reuters
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, Laplace

import requests

import json
from joblib import Parallel, delayed

import math
from time import sleep

In [20]:
# Download do nltk
#nltk.download()

## Importa o dataset de palavras a serem estudadas

In [21]:
# Cria a lista de palavras
words = list(map(str.lower, reuters.words()))
words[0:10]

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan']

In [22]:
# filtra as palavras de acordo com os critérios definidos
freq_reuters = nltk.FreqDist(words)
filtered = list(set(word for word in words if (freq_reuters[word] > 5 and len(word) > 2 and re.fullmatch(r"[a-z]*", word))))

filtered[0: 10]

['inc',
 'heidrun',
 'financially',
 'eighth',
 'dependent',
 'ground',
 'confined',
 'surrounding',
 'lifestyle',
 'moves']

In [23]:
# ordena a lista e salva em um arquivo para evitar uma nova leitura
filtered = sorted(filtered)

with open("storage/good_words.txt", "w") as dump:
    for word in filtered:
        dump.write(f"{word}\n")

## Faz os downloads das paginas da wikipédia para construir o dataset

In [24]:
# Le o bloco de notas com as palavras
with open("storage/good_words.txt", "r") as load:
    good_words = load.read().split("\n")

In [25]:
# Define os parametros para a API
BATCH_SIZE = 50
URL = "https://en.wikipedia.org/w/api.php"
PARAMS = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "main",
    "rvsection": "0",
    "titles": "",
    "format": "json",
}

# Define uma função para dividir os batches
def split_batches(words, batch_size=BATCH_SIZE):
    k = 0
    while k < len(words):
        yield words[k:(k + batch_size)]
        k += batch_size

main_texts = {}
error_log = []

# Inicia a seção
S = requests.Session()

In [26]:
print(f"{len(good_words)} palavras boas para baixar")
print()

# realiza o fetch para cada batch das palavras inicialmente boas
for k, batch in enumerate(split_batches(good_words)):
    try:
        print(f'\rProcessando batch #{k + 1:05}', end='')
        PARAMS['titles'] = '|'.join(batch)
        r = S.get(url=URL, params=PARAMS)
        r_json = r.json()

        # Reverse map of normalized titles.
        title_map = {}
        for item in r_json['query']['normalized']:
            title_map[item['to']] = item['from']
            
        # Get texts.
        texts = {}
        for pageid, page_content in r_json['query']['pages'].items():
            if int(pageid) < 0:
                continue
            text = page_content['revisions'][0]['slots']['main']['*']
            if page_content['title'] in title_map:
                w = title_map[page_content['title']]
            else:
                w = page_content['title']
                
            texts[w] = text

        # Add to global dict.
        main_texts.update(texts)

    except Exception as e:
        error_log.append((e, r))
        
print()
print()
print("Download concluido")

9179 palavras boas para baixar

Processando batch #00184

Download concluido


### Realiza a busca das palavras com redirecionamento

In [27]:
# Fução para limpar as palavras encontradas no redirected
def treat_redirected(text):
    
    text = text.split("[")[-1].split("]")[0].split(" (")[0]
    text = text.strip(" ")
    
    if (re.fullmatch(r"[a-z_ ]*", text.lower())):
        return text
    
    return "Remove Me"

In [28]:
# Procura nos textos recebidos palavras que não tiveram a pagina enontrada
redirected_words = list(set(treat_redirected(main_texts[word]) for word in main_texts.keys() if "#redirect" in main_texts[word][:50].lower()))
try:
    redirect.remove("Remove Me")
except:
    pass

print(f"{len(redirected_words)} palavras redirecionadas para baixar")
print()

# realiza o fetch para cada batch das palavras redirecionadas
for k, batch in enumerate(split_batches(redirected_words)):
    try:
        print(f'\rProcessando batch #{k + 1:05}', end='')
        PARAMS['titles'] = '|'.join(batch)
        r = S.get(url=URL, params=PARAMS)
        r_json = r.json()

        # Reverse map of normalized titles.
        title_map = {}
        for item in r_json['query']['normalized']:
            title_map[item['to']] = item['from']
            
        # Get texts.
        texts = {}
        for pageid, page_content in r_json['query']['pages'].items():
            if int(pageid) < 0:
                continue
            text = page_content['revisions'][0]['slots']['main']['*']
            if page_content['title'] in title_map:
                w = title_map[page_content['title']]
            else:
                w = page_content['title']
                
            texts[w.lower()] = text

        # Add to global dict.
        main_texts.update(texts)

    except Exception as e:
        error_log.append((e, r))
        
print()
print()
print("Download concluido")

2169 palavras redirecionadas para baixar

Processando batch #00044

Download concluido


### Remocão de páginas sem Conteudo

In [29]:
# Retira da lista palavras que foram redirecionadas
words = main_texts.keys()
not_good_data = list(set(word for word in words if "#redirect" in main_texts[word][:50].lower()))
print(f"{len(not_good_data)} paginas redirecionadas para remover")
for word in not_good_data:
    main_texts.pop(word)
    
# Retira palavras que vão para as paginas de desambiguação
words = main_texts.keys()
not_good_data = list(set(word for word in words if "may refer to:" in main_texts[word]))
print(f"{len(not_good_data)} paginas de desambiguação para remover")
for word in not_good_data:
    main_texts.pop(word)
    
# Retira palavras que vão para as paginas muito curtas
words = main_texts.keys()
not_good_data = list(set(word for word in words if "{{Short pages monitor}}" in main_texts[word]))
not_good_data = list(set(not_good_data + [word for word in words if "{{wiktionary redirect}}" in main_texts[word]]))

print(f"{len(not_good_data)} paginas com erro")
for word in not_good_data:
    main_texts.pop(word)

2662 paginas redirecionadas para remover
2616 paginas de desambiguação para remover
30 paginas com erro


In [30]:
# Exporta os resultados para arquivos
with open('storage/texts.json', 'w') as f:
    json.dump(main_texts, f, indent=4)

with open('storage/errors.txt', 'w') as f:
    for e, r in error_log:
        f.write(f'{e} ({type(e)})\nConteudo:\n{r.headers}\n{"*"*100}\n')

## Realia a limpeza dos textos

In [31]:
# Le o bloco de notas com as palavras
with open("storage/texts.json", "r") as load:
    main_texts = json.load(load)

### Demonstração da limpeza

In [32]:
sample = "hanover"

In [33]:
# Define a variavel de teste
clean = main_texts[sample]
clean

'{{about|the German city|other uses|Hanover (disambiguation)}}\n{{Redirect|Hannover}}\n{{short description|City in Lower Saxony, Germany}}\n{{Infobox German location\n|name               = Hanover\n|German_name        = Hannover\n|type               = City\n|image_photo        = {{Photomontage|position=center\n| photo1a = Neues Rathaus Hannover abends.jpg\n| photo2a = HanoverMarktkirche.JPG\n| photo2b = TelemaxHannover.jpg\n| photo3a = Kröpcke_Uhr_Hannover.jpg\n| photo3b = Dschungelpalast.jpg\n| photo4a = Inselgraben_Eilenriede.jpg\n   | size = 280\n   | spacing = 2\n   | color = #FFFFFF\n   | border = 0\n   | foot_montage = {{nobreak|From top: [[New Town Hall (Hanover)|New Town Hall]] of Hanover}}<br/>[[Marktkirche, Hanover|Market Church]], [[Telemax]], Kröpcke-Clock, an [[Asian elephant]] at the [[Hanover Zoo]] and the [[Eilenriede]] forest\n}}\n|image_flag         = Flag of Hanover (city).png\n|image_coa          = Coat of arms of Hannover.svg\n|coordinates        = {{coord|52|22|N|

In [34]:
# Remove referencias do HTML
clean = re.sub(r"<ref.*?(/ref>|/>)", "", clean, flags=re.DOTALL|re.MULTILINE)
clean = re.sub(r"<sup.*?(/sup>|/>)", "", clean, flags=re.DOTALL|re.MULTILINE)
clean

"{{about|the German city|other uses|Hanover (disambiguation)}}\n{{Redirect|Hannover}}\n{{short description|City in Lower Saxony, Germany}}\n{{Infobox German location\n|name               = Hanover\n|German_name        = Hannover\n|type               = City\n|image_photo        = {{Photomontage|position=center\n| photo1a = Neues Rathaus Hannover abends.jpg\n| photo2a = HanoverMarktkirche.JPG\n| photo2b = TelemaxHannover.jpg\n| photo3a = Kröpcke_Uhr_Hannover.jpg\n| photo3b = Dschungelpalast.jpg\n| photo4a = Inselgraben_Eilenriede.jpg\n   | size = 280\n   | spacing = 2\n   | color = #FFFFFF\n   | border = 0\n   | foot_montage = {{nobreak|From top: [[New Town Hall (Hanover)|New Town Hall]] of Hanover}}<br/>[[Marktkirche, Hanover|Market Church]], [[Telemax]], Kröpcke-Clock, an [[Asian elephant]] at the [[Hanover Zoo]] and the [[Eilenriede]] forest\n}}\n|image_flag         = Flag of Hanover (city).png\n|image_coa          = Coat of arms of Hannover.svg\n|coordinates        = {{coord|52|22|N|

In [35]:
# Remove marcas de objetos desenhados na pagina (entre chaves)
clean = re.sub(r"\{\{(?:[^\'\'\'])*?\}\}", "", clean, flags=re.DOTALL|re.MULTILINE)
clean

"\n\n\n<br/>[[Marktkirche, Hanover|Market Church]], [[Telemax]], Kröpcke-Clock, an [[Asian elephant]] at the [[Hanover Zoo]] and the [[Eilenriede]] forest\n}}\n|image_flag         = Flag of Hanover (city).png\n|image_coa          = Coat of arms of Hannover.svg\n|coordinates        = \n|image_plan         = Hannover in H.svg\n|state              = Lower Saxony\n|district           = Hannover\n|elevation          = 55\n|area               = 204.01\n|area_metro         = <!-- Metropolitan area, in km². XXX.XX (no commas or other text) -->\n<!-- |population         = 518386  filled via Gemeindeschlüssel \n|pop_date              = 31 December 2013\n|pop_ref            = \n-->\n|pop_metro          = 1119032\n|postal_code        = 30001 - 30669\n|area_code          = 0511\n|licence            = H\n|Gemeindeschlüssel  = 03 2 41 001\n|NUTS               = <!-- NUTS value: DEXXX  - WILL ''not'' BE DISPLAYED -->\n|LOCODE             = DE HAJ\n|divisions          = 13 districts\n|website          

In [36]:
# Corta as legendas pré texto
splited_clean = clean.split("'''")[1:]

if len(splited_clean) == 0:
    pass
else:
    clean = "".join(splited_clean)

clean

"Hanover or Hannover (;  ; ) is the capital and largest city of the German [[States of Germany|state]] of [[Lower Saxony]]. Its 535,061 (2017) inhabitants make it the [[List of cities in Germany by population|thirteenth-largest city]] in Germany as well as the third-largest city in [[Northern Germany]] after [[Hamburg]] and [[Bremen]]. The city lies at the [[confluence]] of the [[River Leine]] (progression: ) and its [[tributary]] [[Ihme]], in the south of the [[North German Plain]], and is the largest city in the [[Hannover–Braunschweig–Göttingen–Wolfsburg Metropolitan Region]]. It is the fifth-largest city in the [[Low German]] dialect area after Hamburg, [[Dortmund]], [[Essen]] and Bremen.\n\nBefore it became the capital of Lower Saxony in 1946 Hanover was the capital of the [[Principality of Calenberg]] (1636–1692), the [[Electorate of Hanover]] (1692–1814), the [[Kingdom of Hanover]] (1814–1866), the [[Province of Hanover]] of the [[Kingdom of Prussia]] (1868–1918), the [[Province

In [37]:
# Subistitui palavras entre colxete pela própria palavra
clean = re.sub(r"\[\[((?:[^|])*?)\]\]", r"\1", clean, flags=re.DOTALL|re.MULTILINE)
clean

"Hanover or Hannover (;  ; ) is the capital and largest city of the German [[States of Germany|state]] of Lower Saxony. Its 535,061 (2017) inhabitants make it the [[List of cities in Germany by population|thirteenth-largest city]] in Germany as well as the third-largest city in Northern Germany after Hamburg and Bremen. The city lies at the confluence of the River Leine (progression: ) and its tributary Ihme, in the south of the North German Plain, and is the largest city in the Hannover–Braunschweig–Göttingen–Wolfsburg Metropolitan Region. It is the fifth-largest city in the Low German dialect area after Hamburg, Dortmund, Essen and Bremen.\n\nBefore it became the capital of Lower Saxony in 1946 Hanover was the capital of the Principality of Calenberg (1636–1692), the Electorate of Hanover (1692–1814), the Kingdom of Hanover (1814–1866), the Province of Hanover of the Kingdom of Prussia (1868–1918), the Province of Hanover of the Free State of Prussia (1918–1946) and of the State of H

In [38]:
# Subistitui palavras entre colxete com pipe pela palavra depois do pipe
clean = re.sub(r"\[\[(?:[^|]|)*(.*?)\]\]", r"\1", clean, flags=re.DOTALL|re.MULTILINE)
clean

"Hanover or Hannover (;  ; ) is the capital and largest city of the German |state of Lower Saxony. Its 535,061 (2017) inhabitants make it the |thirteenth-largest city in Germany as well as the third-largest city in Northern Germany after Hamburg and Bremen. The city lies at the confluence of the River Leine (progression: ) and its tributary Ihme, in the south of the North German Plain, and is the largest city in the Hannover–Braunschweig–Göttingen–Wolfsburg Metropolitan Region. It is the fifth-largest city in the Low German dialect area after Hamburg, Dortmund, Essen and Bremen.\n\nBefore it became the capital of Lower Saxony in 1946 Hanover was the capital of the Principality of Calenberg (1636–1692), the Electorate of Hanover (1692–1814), the Kingdom of Hanover (1814–1866), the Province of Hanover of the Kingdom of Prussia (1868–1918), the Province of Hanover of the Free State of Prussia (1918–1946) and of the State of Hanover (1946). From 1714 to 1837 Hanover was by personal union t

In [39]:
# limpa algumas outras variaveis irrelevantes
clean = re.sub(r"\n|\'", r"", clean, flags=re.DOTALL|re.MULTILINE)
clean = re.sub(r"}}", r"", clean, flags=re.DOTALL|re.MULTILINE)
clean = re.sub(r"\|", r"", clean, flags=re.DOTALL|re.MULTILINE)
clean = re.sub(r"\(;|\(,", r"(SPECIALCHAR", clean, flags=re.DOTALL|re.MULTILINE)
clean = re.sub(r"\(SPECIALCHAR.*?\)", "", clean, flags=re.DOTALL|re.MULTILINE)
clean

'Hanover or Hannover  is the capital and largest city of the German state of Lower Saxony. Its 535,061 (2017) inhabitants make it the thirteenth-largest city in Germany as well as the third-largest city in Northern Germany after Hamburg and Bremen. The city lies at the confluence of the River Leine (progression: ) and its tributary Ihme, in the south of the North German Plain, and is the largest city in the Hannover–Braunschweig–Göttingen–Wolfsburg Metropolitan Region. It is the fifth-largest city in the Low German dialect area after Hamburg, Dortmund, Essen and Bremen.Before it became the capital of Lower Saxony in 1946 Hanover was the capital of the Principality of Calenberg (1636–1692), the Electorate of Hanover (1692–1814), the Kingdom of Hanover (1814–1866), the Province of Hanover of the Kingdom of Prussia (1868–1918), the Province of Hanover of the Free State of Prussia (1918–1946) and of the State of Hanover (1946). From 1714 to 1837 Hanover was by personal union the family sea

### Limpeza total

In [40]:
def clean_string(clean, remove_special=True):
    # Remove referencias do HTML
    clean = re.sub(r"<ref.*?(/ref>|/>)", "", clean, flags=re.DOTALL|re.MULTILINE)
    clean = re.sub(r"<sup.*?(/sup>|/>)", "", clean, flags=re.DOTALL|re.MULTILINE)

    # Remove marcas de objetos desenhados na pagina (entre chaves)
    clean = re.sub(r"\{\{(?:[^\'\'\'])*?\}\}", "", clean, flags=re.DOTALL|re.MULTILINE)

    # Corta as legendas pré texto
    splited_clean = clean.split("'''")[1:]

    if len(splited_clean) == 0:
        pass
    else:
        clean = "".join(splited_clean)

    # Subistitui palavras entre colchete pela própria palavra
    clean = re.sub(r"\[\[((?:[^|])*?)\]\]", r"\1", clean, flags=re.DOTALL|re.MULTILINE)

    # Subistitui palavras entre colxete com pipe pela palavra depois do pipe
    clean = re.sub(r"\[\[(?:[^|]|)*(.*?)\]\]", r"\1", clean, flags=re.DOTALL|re.MULTILINE)

    # limpa algumas outras variaveis irrelevantes
    clean = re.sub(r"\n|\'", r"", clean, flags=re.DOTALL|re.MULTILINE)
    clean = re.sub(r"}}", r"", clean, flags=re.DOTALL|re.MULTILINE)
    clean = re.sub(r"\|", r"", clean, flags=re.DOTALL|re.MULTILINE)
    clean = re.sub(r"\(;|\(,", r"(SPECIALCHAR", clean, flags=re.DOTALL|re.MULTILINE)
    
    if remove_special:
        clean = re.sub(r"\(SPECIALCHAR.*?\)", "", clean, flags=re.DOTALL|re.MULTILINE)
    
    return clean

In [41]:
for word in list(main_texts.keys()):
    
    new_text = clean_string(main_texts[word])
    
    if len(new_text) == 0:
        main_texts.pop(word)
    else:
        main_texts[word] = new_text

In [42]:
# Exporta os resultados para arquivos
with open('storage/clean_texts.json', 'w') as f:
    json.dump(main_texts, f, indent=4)

## Divide o texto em sentenças usando o tokenizador Punkt

In [43]:
# Le o bloco de notas com as palavras
with open("storage/clean_texts.json", "r") as load:
    main_texts = json.load(load)

In [44]:
sample = "hanover"

In [45]:
main_sents = {}
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
for key, value in main_texts.items():
    main_sents[key] = sent_tokenizer.tokenize(value)

In [46]:
main_sents[sample]

['Hanover or Hannover  is the capital and largest city of the German state of Lower Saxony.',
 'Its 535,061 (2017) inhabitants make it the thirteenth-largest city in Germany as well as the third-largest city in Northern Germany after Hamburg and Bremen.',
 'The city lies at the confluence of the River Leine (progression: ) and its tributary Ihme, in the south of the North German Plain, and is the largest city in the Hannover–Braunschweig–Göttingen–Wolfsburg Metropolitan Region.',
 'It is the fifth-largest city in the Low German dialect area after Hamburg, Dortmund, Essen and Bremen.Before it became the capital of Lower Saxony in 1946 Hanover was the capital of the Principality of Calenberg (1636–1692), the Electorate of Hanover (1692–1814), the Kingdom of Hanover (1814–1866), the Province of Hanover of the Kingdom of Prussia (1868–1918), the Province of Hanover of the Free State of Prussia (1918–1946) and of the State of Hanover (1946).',
 'From 1714 to 1837 Hanover was by personal uni

In [47]:
# Exporta os resultados para arquivos
with open('storage/def_sents.json', 'w') as f:
    json.dump(main_sents, f, indent=4)

## Análise por Perplexidade

A fim de selecionar a melhor setença que define cada palavra-chave, será utilizada a análise por perplexidade, método este que consiste em mensurar a "estranheza", através da perspectiva de um modelo treinado sob uma base grande de textos, de cada sentença.

In [48]:
# Le o bloco de notas com as palavras
with open("storage/def_sents.json", "r") as load:
    main_sents = json.load(load)

### Usando Corpus Reuters como base

Nesta seção o corpus reuters é utilizado como base de treinamento para o modelo, com o pressuposto inicial de que ao utilizar um conjunto grande de textos, tem-se um modelo amplamente treinado sobre palavras muito utilizadas e pouco utilizadas para mensurar o grau de "estranhesa" de cada sentença.

In [49]:
# Prepraração dos dados
reuters_sentences = reuters.sents()
reuters_train, reuters_vocab = padded_everygram_pipeline(2, reuters_sentences)

reuters_train = list(list(t) for t in reuters_train)
reuters_vocab = list(reuters_vocab)

#Treinamento do Modelo
lm_reuters = Laplace(2)
lm_reuters.fit(reuters_train, reuters_vocab)

In [50]:
sample = "hanover"

In [51]:
# Encontra as perplexidades da amostra
text_sentences = [t.split() for t in main_sents[sample]]
test, _ = padded_everygram_pipeline(2, text_sentences)
test = list(list(t) for t in test)

import time 
start_time = time.time()

# Calcula a perplexidade das sentenças
idx = 0 
min_value = math.inf
for i, s in enumerate(test):
    px = lm_reuters.perplexity(s)
    
    if(min_value > px):
        min_value = px
        idx = i

    print(px)
print(f"Tempo de Execução: {time.time() - start_time} segundos")

3430.347882606758
8372.738449782717
3529.6318222835257
7686.65569545711
10478.996008500379
14306.610566047464
13130.070098942655
7638.787064473866
11221.220325754444
5488.853729160141
19418.311305672054
10551.84100606817
4051.3891598621776
5014.145133997854
Tempo de Execução: 32.64062285423279 segundos


Como pode-se observar nos testes utilizando o corpus router como base, há um valor alto de tempo de execução para apenas uma palavra-chave, o que torna a execução do codigo para todo o conjunto de pavras da enciclopedia a ser implementada muito alto, aproximadamente de 1:30h para o conjunto atual, logo o teste completo com esta base, embora implementado, n foi executado neste notebook. 

In [52]:
def find_lowest_perplexity_reuter_parallel(main_sents, words, do_print=False):
    
    encyclopedia_reuters = {}
    for i, word in enumerate(words):
        idx, min_value = find_lowest_perplexity_reuters(main_sents, word)
        encyclopedia_reuters[word] = main_sents[word][idx]
        
    return encyclopedia_reuters
        
def find_lowest_perplexity_reuters(main_sents, word, do_print=False):

    # Prepraração dos dados
    text_sentences = [t.split() for t in main_sents[word]]
    test, _ = padded_everygram_pipeline(2, text_sentences)
    test = list(list(t) for t in test)

    # Calcula a perplexidade das sentenças
    idx = 0 
    min_value = math.inf
    for i, s in enumerate(test):
        px = lm_reuters.perplexity(s)

        if(min_value > px):
            min_value = px
            idx = i

        if do_print:
            print(px)
            
    return idx, min_value 

#### Rodando em serie

In [67]:
# encyclopedia_reuters = {}
# words = list(main_sents.keys())
# total = len(words)
# for i, word in enumerate(words):
#     idx, min_value = find_lowest_perplexity_reuters(main_sents, word)
#     encyclopedia_reuters[word] = main_sents[word][idx]
    
#     print(f"\r{i+1} out of {total}", end='')

#### Rodando em Paralelo

In [ ]:
# n_jobs = 12
# encyclopedia_reuters = {}
# words = list(main_sents.keys())
# part = round(len(words)/n_jobs)

# words_batch = []
# for i in range(n_jobs):
#     if ((i+1)*part > len(words)):
#         words_batch.append(words[i*part:])
#     else:
#         words_batch.append(words[i*part:(i+1)*part])

In [ ]:
# encyclopedia_reuters = {}
# results = Parallel(n_jobs=n_jobs, verbose=50)(delayed(find_lowest_perplexity_reuter_parallel)(main_sents, words) for words in words_batch)

In [ ]:
# # Exporta os resultados para arquivos
# with open('storage/encyclopedia_reuters.json', 'w') as f:
#     json.dump(encyclopedia_reuters, f, indent=4)

### Usando o próprio texto como base

Nesta seção o próprio documento referente a palavra-chave é utilizado como base de treinamento para o modelo, com o pressuposto inicial de que a sentença que melhor define a palavra contem elementos com uso mais frequente em seu contexto específico.

In [53]:
sample = "hanover"

In [54]:
# Prepraração dos dados
text_sentences = [t.split() for t in main_sents[sample]]
train, vocab = padded_everygram_pipeline(2, text_sentences)

train = list(list(t) for t in train)
vocab = list(vocab)

# Treinamento do Modelo
lm = Laplace(2)
lm.fit(train, vocab)

# Calcula a perplexidade das sentenças
idx = 0 
min_value = math.inf
for i, s in enumerate(train):
    px = lm.perplexity(s)
    
    if(min_value > px):
        min_value = px
        idx = i

    print(px)

79.7550009448657
124.01750308067002
93.11147167318235
83.93973690163187
121.94755697767927
110.54408863750604
128.84281222783548
116.52755746298136
150.0887349015754
113.86013794070298
106.94385975498913
117.51953388139117
114.44498585763819
123.91459807786784


In [55]:
def find_lowest_perplexity_same_parallel(main_sents, words, do_print=False):
    
    encyclopedia_same = {}
    for i, word in enumerate(words):
        idx, min_value = find_lowest_perplexity_same(main_sents, word)
        encyclopedia_same[word] = main_sents[word][idx]
        
    return encyclopedia_same

def find_lowest_perplexity_same(main_sents, word, do_print=False):

    # Prepraração dos dados
    text_sentences = [t.split() for t in main_sents[word]]
    train, vocab = padded_everygram_pipeline(2, text_sentences)

    train = list(list(t) for t in train)
    vocab = list(vocab)

    # Treinamento do Modelo
    lm = Laplace(2)
    lm.fit(train, vocab)
    
    # Calcula a perplexidade das sentenças
    idx = 0 
    min_value = math.inf
    for i, s in enumerate(train):
        px = lm.perplexity(s)
        
        if(min_value > px):
            min_value = px
            idx = i
        
        if do_print:
            print(px)
            
    return idx, min_value 

In [56]:
encyclopedia_same = {}
words = list(main_sents.keys())
total = len(words)
for i, word in enumerate(words):
    idx, min_value = find_lowest_perplexity_same(main_sents, word)
    encyclopedia_same[word] = main_sents[word][idx]
    
    print(f"\r{i+1} out of {total}", end='')

3299 out of 3299

#### Rodando em paralelo

In [57]:
n_jobs = 12
encyclopedia_reuters = {}
words = list(main_sents.keys())
part = round(len(words)/n_jobs)

words_batch = []
for i in range(n_jobs):
    if ((i+1)*part > len(words)):
        words_batch.append(words[i*part:])
    else:
        words_batch.append(words[i*part:(i+1)*part])

In [58]:
encyclopedia_same = {}
results = Parallel(n_jobs=n_jobs, verbose=50)(delayed(find_lowest_perplexity_same_parallel)(main_sents, words) for words in words_batch)

for result in results:
    encyclopedia_same.update(result)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:  1.4min
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:  1.4min remaining:  7.1min
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:  1.5min remaining:  4.6min
[Parallel(n_jobs=12)]: Done   4 out of  12 | elapsed:  1.6min remaining:  3.1min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:  1.6min remaining:  2.2min
[Parallel(n_jobs=12)]: Done   6 out of  12 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:  1.8min remaining:  1.3min
[Parallel(n_jobs=12)]: Done   8 out of  12 | elapsed:  1.9min remaining:   55.7s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:  1.9min remaining:   37.6s
[Parallel(n_jobs=12)]: Done  10 out of  12 | elapsed:  2.2min remaining:   25.9s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapse

In [59]:
# Exporta os resultados para arquivos
with open('storage/encyclopedia_same.json', 'w') as f:
    json.dump(encyclopedia_same, f, indent=4)

## Explora as encyclopedias

In [61]:
# Le o bloco de notas com os dados
with open("storage/encyclopedia_same.json", "r") as load:
    encyclopedia_same = json.load(load)
    
#with open("storage/encyclopedia_reuters.json", "r") as load:
#    encyclopedia_reuters = json.load(load)

In [62]:
#encyclopedia_reuters

In [63]:
encyclopedia_same

{'abbett': 'Abbett is a surname.',
 'abdul': 'Abdul (also transliterated as Abdal, Abdur, Abdelx, Abdil, Abdol, Abdool, or Abdoul; , {{translarDINʿAbd al-) is the most frequent transliteration of the combination of the Arabic word Abd  and the definite prefix al / el .It is the initial component of many compound names, names made of two words.',
 'ablaze': 'The film uses stock footage from two other films.',
 'acceleration': 'If the speed of the car decreases, this is an acceleration in the opposite direction of the velocity of the vehicle, sometimes called deceleration or Retrograde burning in spacecraft.',
 'acceptance': 'Acceptance in human psychology is a persons assent to the reality of a situation, recognizing a process or condition (often a negative or uncomfortable situation) without attempting to change it or protest it.',
 'accepted': 'The story takes place in Wickliffe and a fictitious college town called Harmon in Ohio.',
 'accession': 'Accession refers to the general idea 

In [64]:
encyclopedia_same["hanover"]

'Hanover or Hannover  is the capital and largest city of the German state of Lower Saxony.'

## Conclusão

Em posse da enciclopédia, construída sobre o modelo treinado com o próprio contexto de cada palavra-chave, pode-se observar que em grande parte, as descrições são sentenças pertinentes, porém há palavras com descrições inadequadas, isso pode ter ocorrido devido a essas palavras terem um documento com muito pouco conteúdo escrito, prejudicando o treinamento do modelo para a análise de perplexidade.